# Stable Diffusion 3

In [1]:
import torch
import random
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# Static seed for reproducibility or turn on randomization
RANDOM_SEED = 42
RANDOMIZE_SEED = False

# Check if we should use a static seed, if not randomize it
if RANDOMIZE_SEED:
    RANDOM_SEED = random.randint(0, 1000000000)
    print(f"Using static seed: {RANDOM_SEED}")

# Device setup: Try to use xpu, fallback to cuda, then cpu
device = "xpu" if torch.xpu.is_available() else "cpu"
print(f"Using device: {device}")

# User-definable parameters with defaults
parameters = {
    "prompt": "Hyperrealistic. A beautiful cyberpunk woman in a futuristic city, surrounded by neon lights and rain. The background features towering skyscrapers with vibrant neon signs, futuristic vehicles, and wet streets reflecting the city lights. The atmosphere is moody with a dark, rainy night, creating a cinematic, sci-fi look.",
    "negative_prompt": "Low quality, blurry, unrealistic, poorly lit, lack of detail, no futuristic elements, unattractive, low-resolution, no rain, no neon lights, no cyberpunk elements.",
    "num_inference_steps": 50,  # Number of denoising steps
    "guidance_scale": 14.0,  # Adherence to prompt
    "width": 512,  # Image width
    "height": 512,  # Image height
    "seed": RANDOM_SEED,  # Seed for reproducibility
    "device": device,  # Device for computation
    "use_amp": True,  # Automatic mixed precision
}

# Disable gradients since we are not training
torch.set_grad_enabled(False)

# Configure random seed
generator = torch.manual_seed(parameters["seed"])

# Load the pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-0", 
    torch_dtype=torch.float32 if parameters["device"] == "cpu" else torch.float16
)

# Load and set the DPM++ 2M scheduler
pipe.scheduler = DPMSolverMultistepScheduler.from_pretrained(
    "stabilityai/stable-diffusion-3-0", 
    subfolder="scheduler"
)

# Move pipeline to the desired device
pipe = pipe.to(parameters["device"])

# Generate the image
with torch.autocast(parameters["device"]) if parameters["use_amp"] else torch.no_grad():
    image = pipe(
        prompt=parameters["prompt"],
        negative_prompt=parameters["negative_prompt"],
        num_inference_steps=parameters["num_inference_steps"],
        guidance_scale=parameters["guidance_scale"],
        height=parameters["height"],
        width=parameters["width"],
        generator=generator,
    ).images[0]

# Display the image
image

c:\Users\user\miniconda3\envs\xpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Couldn't connect to the Hub: 401 Client Error. (Request ID: Root=1-6786166a-48563aad459adfbf5d0c3823;4081c5fa-57f8-49f7-aa50-89c0bf90e4ad)

Repository Not Found for url: https://huggingface.co/api/models/stabilityai/stable-diffusion-3-0.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password..
Will try to load from local cache.


Using device: xpu


OSError: Cannot load model stabilityai/stable-diffusion-3-0: model is not cached locally and an error occurred while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.

: 